In [ ]:
import os
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory to the workshop directory
# which was created in your username directory under /scratch/vp91
os.chdir(os.path.expandvars("/scratch/vp91/$USER/"))

# Datasets and grouping 

The functionality we have covered so far is fantastic, but what if we want to group arrays together in some kind of coherent way?

Don't worry `xarray` has us covered with a `Dataset`. Lets make a few arrays and combine them in a dataset.

In [ ]:
import numpy as np

## One dimensional example

We are going to start with a dataset that shares a single dimension just to get the hang of how datasets work.

Our problem statement is that have several harmonic oscillator, modelling several springs, and we want to keep track of their motion over time. The motion of a harmonic oscillator is given by:

$ x(t) = A cos({\omega t + \phi}) $

Where $A$ is the amplitude, $\omega$ is the angular frequency and $\phi$ is the phase shift. These values are determined by the initial positions and velocities of the spring as well as the mass and stiffness of  the spring, but lets not get too into that, the primary purpose here is to learn how we are going to deal with our dataset using xarray!

First lets define a function that propogates our harmonic oscillator over time:

In [ ]:
def harmonic(A,t,omega,phi):
    x_t = A*np.cos(omega*t + phi)
    return x_t

Okay now lets make a time domain for us to propogate over

In [ ]:
npoints = 1000
time_domain = xr.DataArray(np.arange(npoints), dims=("t"), coords={"t":np.arange(npoints)})
time_domain

In [ ]:
#initial conditions
A = 100
omega = 0.1
phi = 0.1


Once again we use the very handy `xr.apply_ufunc` to apply our function in a vectorised way accross our domain.

In [ ]:
result = xr.apply_ufunc(harmonic,A,time_domain.t,omega,phi)
result.plot()

### Challenge

Try with some different intitial conditions.

Okay cool! That looks a lot like harmonic oscillation. Now lets make a **collection** of them with random intial conditions!

In [ ]:
oscillators = {}

n_oscillators = 100

for i in range(n_oscillators):
    
    #initial conditions
    A = np.random.rand()*100
    omega = np.random.rand()
    phi = np.random.rand()
    
    # apply over array
    result = xr.apply_ufunc(harmonic,A,time_domain.t,omega,phi)
    
    # add metadata
    result.attrs["A"] = A
    result.attrs["phi"] = phi
    result.attrs["omega"] = omega
    
    tag = "oscillator_" + str(i)
    oscillators[tag] = result

Lets plot an oscillator in our dictionary of oscillators

In [ ]:
oscillators["oscillator_6"].plot()

Okay this is good but our oscillators actually the **same time axis**. This is the core of the xarray dataset concept, arrays with the same **named dimensions** share a coordinate system. 

Lets turn our `dict` of oscillators into a `DataSet` and play around with it a bit. Make sure to expand the `Data Variables` tab to have a look at our 100 oscillators.

In [ ]:
dataset = xr.Dataset(data_vars=oscillators)
dataset

In [ ]:
dataset["oscillator_6"] # a dataset is a dictionary-like container

Note that we recorded the values of our variables in our model as `attrs` on each `DataArray`

In [ ]:
dataset.oscillator_6.plot()

Ok cool, we can play with our dataset nicely.

Now lets use the power of the `Dataset` and its shared coordinate axes. We are going to find the largest oscillator positions away from the equilibrium position (0) accross **all of our oscillators**.

In [ ]:
# works across our 100 arrays
np.abs(dataset).max()

### Challenge

Perform a transformation on our dataset by taking the `sin` of our dataset and plotting an oscillator

In [ ]:
# Perform a transformation on our dataset by taking the sin of our dataset and plotting an oscillator

<details><summary><b>Solution</b></summary>
   <pre>
    <br> np.sin(dataset).oscillator_1.plot()
   </pre>
</details>

Awesome work! We now know how to work with a `Dataset`, even if this one is rather simple and only has one shared dimension. We will soon work with one that has more complexity and more shared dimensions. 

We can save our dataset to a netcdf file using a similar API to what we used for a `DataArray`

In [ ]:
dataset.to_netcdf("oscillators.nc")

In [ ]:
reloaded = xr.open_dataset("oscillators.nc")

In [ ]:
reloaded["oscillator_6"]

## Conclusion

Now you know how to use a `Dataset`! We are going to look at a more complex dataset example now.

